In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np

In [9]:
def infoparse(x):
    key   = [i.split('=')[0] for i in x.split(';')]
    value = [i.split('=')[1] for i in x.split(';')]
    return dict(zip(key,value))

In [2]:
def rev_comp(strSeq):
	dicComp = {'A':'T','T':'A','G':'C','C':'G','N':'N'}
	strCseq = ''
	for i in strSeq:
		try:
			strCseq += dicComp[i.upper()]
		except KeyError:
			strCseq += 'N'
	# End of for i
	return(strCseq[::-1])

In [3]:
file_fa = '../Ahal.assembly.scf.fasta'

bulk = open(file_fa).read()
bulk = bulk.split('>')
dicHD2seq = {}
for each_bulk in bulk:
    if each_bulk.strip() == '':
        continue
    genename = each_bulk.split('\n')[0]
    seq      = ''.join(each_bulk.split('\n')[1:])
    dicHD2seq[genename] = seq

In [20]:
file_gff = '../Ahal.assembly.genepred.gff3.final.gff3'

df_gff = pd.read_csv(file_gff,sep='\t',header=None,comment='#')

mask        = (df_gff[2] == 'gene')
df_gff_gene = df_gff[mask]      
df_gff_gene['genename'] = df_gff_gene[8].apply(lambda x:infoparse(x)['ID'])

mask        = (df_gff[2] == 'mRNA')
df_gff_mRNA = df_gff[mask]
mRNA_ID     = df_gff_mRNA[8].apply(lambda x : infoparse(x)['ID'])
mRNA_Parent = df_gff_mRNA[8].apply(lambda x : infoparse(x)['Parent'])
dict2g = dict(zip(mRNA_ID,mRNA_Parent))

mask = (df_gff[2] == 'CDS')
df_gff_cds = df_gff[mask]      

df_gff_cds['genename'] = df_gff_cds[8].apply(lambda x : infoparse(x)['Parent'])

df_gff_cds_index = df_gff_cds.set_index('genename')

df_gff_cds_index = df_gff_cds_index.sort([3], ascending=[1])

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:22: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [18]:
df_gff_cds.head()

,0,1,2,3,4,5,6,7,8,genename
2,000430F-pilon,AUGUSTUS,CDS,1,82,.,+,2,ID=CDS1;Parent=mRNA1,mRNA1
4,000430F-pilon,AUGUSTUS,CDS,174,490,.,+,1,ID=CDS1;Parent=mRNA1,mRNA1
6,000430F-pilon,AUGUSTUS,CDS,1179,1324,.,+,2,ID=CDS1;Parent=mRNA1,mRNA1
8,000430F-pilon,AUGUSTUS,CDS,1418,1552,.,+,0,ID=CDS1;Parent=mRNA1,mRNA1
12,000430F-pilon,AUGUSTUS,CDS,7904,8055,.,+,0,ID=CDS2;Parent=mRNA2,mRNA2


In [14]:
i = 0
dic = {'transcriptname' : [],
       'strand' : [],
       'CDSloc' : [],
       'CDSseq' : []}
for genename in set(df_gff_cds_index.index):
    
    dic['transcriptname'].append(genename)
    
    edf = df_gff_cds_index.loc[genename]
    if str(type(edf)) == "<class 'pandas.core.frame.DataFrame'>":
        CDS_list = zip(list(edf[3]),list(edf[4]))
        chromosome = edf[0][0]
        strand     = edf[6][0]
    else:
        CDS_list = zip([edf[3]],[edf[4]])
        chromosome = edf[0]
        strand     = edf[6]
    
    dic['strand'].append(strand)
    cdsseq = ''
    for l,r in CDS_list:
        #print l,r
        cdsseq += dicHD2seq[chromosome][l-1:r]
    if strand == '+':
        pass
    else: 
        cdsseq = rev_comp(cdsseq)
    dic['CDSloc'].append(CDS_list)
    dic['CDSseq'].append(cdsseq)
    #print genename,chromosome,strand,CDS_list,cdsseq
    #if i == 10:
    #    break
    #i += 1

In [23]:
df_cds = pd.DataFrame(dic)

In [24]:
df_cds['gene'] = df_cds['transcriptname'].apply(lambda x : dict2g[x])

In [25]:
df_cds.head()

,CDSloc,CDSseq,strand,transcriptname,gene
0,"[(764493, 764747)]",ATGAAGGAAGAAGATAGAGTTGGTTCATCGTCGTCTTCTTCATCTT...,-,mRNA35077,gene35077
1,"[(497435, 498769)]",ATGAAAATCTTTGATGTGAGTTTCACCGGAGAGTTCATAGTCAAAG...,-,mRNA11070,gene11070
2,"[(499948, 500136), (500245, 500392), (500470, ...",ATGGTAATGATGATTTTGTGTGGTAATGAGAATTTTGGGTTTCCAG...,-,mRNA11071,gene11071
3,"[(504929, 505371), (505496, 506075)]",ATGGAGAAGTCAGGAGAAACTCCAAAATGGGAAACTCTCGACAGAA...,+,mRNA11072,gene11072
4,"[(507015, 507345), (507540, 507556)]",ATGGCTCATTTATCATGGTATGAACAAGGAGTTTTGAGAGATGATG...,-,mRNA11073,gene11073


In [26]:
df_cds_index = df_cds.set_index(['gene','transcriptname'])

In [27]:
df_cds_index.to_pickle(file_gff+'.cdsseq.pk')

In [28]:
df_cds_index.head()

,,CDSloc,CDSseq,strand
gene,transcriptname,,,
gene35077,mRNA35077,"[(764493, 764747)]",ATGAAGGAAGAAGATAGAGTTGGTTCATCGTCGTCTTCTTCATCTT...,-
gene11070,mRNA11070,"[(497435, 498769)]",ATGAAAATCTTTGATGTGAGTTTCACCGGAGAGTTCATAGTCAAAG...,-
gene11071,mRNA11071,"[(499948, 500136), (500245, 500392), (500470, ...",ATGGTAATGATGATTTTGTGTGGTAATGAGAATTTTGGGTTTCCAG...,-
gene11072,mRNA11072,"[(504929, 505371), (505496, 506075)]",ATGGAGAAGTCAGGAGAAACTCCAAAATGGGAAACTCTCGACAGAA...,+
gene11073,mRNA11073,"[(507015, 507345), (507540, 507556)]",ATGGCTCATTTATCATGGTATGAACAAGGAGTTTTGAGAGATGATG...,-


In [ ]:
with open('cds.fa','w') as f:
    for ix in df_cds_index.index:
        print (ix)
        sub_df = df_cds_index.loc[ix]
        header = ix[1]
        seq    = sub_df['CDSseq']
        print('>'+header,file=f)
        print(seq,file=f)


('gene35077', 'mRNA35077')
('gene11070', 'mRNA11070')
('gene11071', 'mRNA11071')
('gene11072', 'mRNA11072')
('gene11073', 'mRNA11073')
('gene11074', 'mRNA11074')
('gene11075', 'mRNA11075')
('gene11076', 'mRNA11076')
('gene11077', 'mRNA11077')
('gene11078', 'mRNA11078')
('gene11079', 'mRNA11079')
('gene8080', 'mRNA8080')
('gene8081', 'mRNA8081')
('gene8086', 'mRNA8086')
('gene8087', 'mRNA8087')
('gene8084', 'mRNA8084')
('gene8085', 'mRNA8085')
('gene2475', 'mRNA2475')
('gene29317', 'mRNA29317')
('gene34509', 'mRNA34509')
('gene34508', 'mRNA34508')
('gene34505', 'mRNA34505')
('gene34504', 'mRNA34504')
('gene34507', 'mRNA34507')
('gene34506', 'mRNA34506')
('gene34501', 'mRNA34501')
('gene34500', 'mRNA34500')
('gene34503', 'mRNA34503')
('gene34502', 'mRNA34502')
('gene29314', 'mRNA29314')
('gene5872', 'mRNA5872')
('gene8088', 'mRNA8088')
('gene29315', 'mRNA29315')
('gene18526', 'mRNA18526')
('gene25338', 'mRNA25338')
('gene25339', 'mRNA25339')
('gene25330', 'mRNA25330')
('gene25331', 'mRNA